In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Stats import WeightQuantization,IntroduceFaultsInWeights,GenerateFaultsList
from Nets import GetNeuralNetworkModel
from Training import GetDatasets
from Simulation import get_all_outputs

Base


In [ ]:
Estudio d elos pesos

In [2]:


from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds

### Definiciones

In [3]:
# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4
# Tamaño del buffer de pesos == al tamaño de la capa con mayor numero de pesos (885120 pesos de 16 bits cada uno)
#wbuffer_size = 885120*word_size
# Tamaño del buffer de activaciones == al tamaño de la capa con mayor numero de activaciones (290400 pesos de 16 bits cada uno)
#abuffer_size = 290400*word_size
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

### Generacion de una muestra de fallos

### Cargar el dataset

In [4]:
trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)

C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


In [6]:
error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/error_mask_054')
locs = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_x/error_mask_x/vc_707/locs_054')


activation_aging = [False]*11

afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4

Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
num_weights=[]
num_fallos=[]
num_direc_afect= []
name_layer=[]
layer_size=[]
acc_layer=[]

for index,layer in enumerate(Net2.layers):
    print(index)
    weights = layer.get_weights()
    
    
    if weights:
        layer_name=Net2.layers[index].__class__.__name__
        print(layer_name)
        
        name_layer.append(layer_name)
        
        
        vectorized_pesos = []
            
        for itm in weights:
            #print('    - Numero de pesos x itm:',itm.size)
            vectorized_pesos.extend(itm.flatten(order='F'))
            print('    - Tamaño del itm vectorizado',len(vectorized_pesos))
        vectorized_pesos = np.array(vectorized_pesos)
        num_weights.append(vectorized_pesos.size)
        print('  Numero total de pesos:',vectorized_pesos.size)
            
        indexList,faultList,NumberOfFaults = GenerateFaultsList(shape=vectorized_pesos.shape,locs=locs,error_mask=error_mask)
        num_fallos.append(NumberOfFaults)
        print('  Total de numero de fallos:',NumberOfFaults)
        #print('  indexList:',len(indexList))
        #print('  Listfault fallos:',len(faultList))
      
        if NumberOfFaults >  0 :
            print('1dentro',NumberOfFaults)
            weight_fail= IntroduceFaultsInWeights(vectorized_pesos,indexList,faultList,wint_size,wfrac_size) 
            direc_afect= np.count_nonzero(vectorized_pesos!=weight_fail)
            num_direc_afect.append(direc_afect)
        if NumberOfFaults==0:
            print('dentro',NumberOfFaults)
            num_direc_afect.append(0)
            print('  Total de direcciones afectadas:',np.count_nonzero(vectorized_pesos!=weight_fail))
            print('num_direc_afect',num_direc_afect)
        for index, weight in enumerate(weights):
            print('weight.size',weight.size)
            
            weights[index] = weight_fail[0:weight.size].numpy().reshape(weight.shape,order='F')
        layer.set_weights(weights)
        loss = tf.keras.losses.CategoricalCrossentropy()
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
        loss,acc  = Net2.evaluate(test_dataset) 
        acc_layer.append(acc)
        layer.get_weights()
        for index, weight in enumerate(weights):
            print('weight.size',weight.size)
            print(type(vectorized_pesos))
            weight[index]= vectorized_pesos[0:weight.size].reshape(weight.shape,order='F')
            print(index)
        layer.set_weights(weights)
            
                                          
DFrame_num_weights= pd.DataFrame(num_weights)
DFrame_num_fallos= pd.DataFrame(num_fallos)
DFrame_num_direc_afect= pd.DataFrame(num_direc_afect)
DFrame_layer= pd.DataFrame(name_layer)
DFrame_acc_layer= pd.DataFrame(acc_layer)

                                          

statistics = pd.concat([DFrame_layer,DFrame_num_weights,DFrame_num_fallos,DFrame_num_direc_afect,DFrame_acc_layer], axis=1, join='outer')
statistics.columns =['DFrame_layer', 'num_weights','num_fallos', 'num_direc_afect','Acc']
print(statistics)
statistics.to_excel('statist_weigts.xlsx', sheet_name='fichero_707', index=False)                                          


0
1
2
3
Conv2D
    - Tamaño del itm vectorizado 34848
    - Tamaño del itm vectorizado 34944
  Numero total de pesos: 34944
  Total de numero de fallos: 134
1dentro 134
inyectando errores en los pesos
weight.size 34848
weight.size 96
47/47 [==============================] - 14s 306ms/step - loss: 13.1099 - accuracy: 0.1227
weight.size 34848
<class 'numpy.ndarray'>


ValueError: could not broadcast input array from shape (11,11,3,96) into shape (11,3,96)

## Algoritmo

In [ ]:
Algoritmo(crear una función que tome el modelo)
le pido los pesos al modelo
recorro las el modelo por capas
si la capa tiene pesos y es convolución:
    calculo la candidad de elementos que tiene la variable weights.size
    lo guardo en un variable
    la vectorizo a 1 dimensión y a concateno con el bias 
    le paso ese itm a la función IntroduceFaultsInWeights
    luego la separos y las coloco son el shape original 
    y a model.layers[index] le establezco los pesos ya con fallos
si la capa tiene pesos y es Normalizatión:
    tiene los parametros gamma,beta,moving_mean,moving_std
    hago lo mismo por cada elemento
    (seria mejor hacer dos funciones nuevas para que hagan lo de vectorizar y luego separar de nuevo)
    model.layers[index] le establezco los pesos ya con fallos
Si es un capa Dense:
    tiene weights y bias
    hago los mismo 
    model.layers[index] le establezco los pesos ya con fallos
la función devuelve el modelo para qu econtinue el algoritmo   

para comprobar que funciona imprimo las variables positionList,faultLis




## Función

### Uso la misma forma que usas tu para llevarlo a 1 dimensión pero por separado igual
### hay break en el código porque fui comparando los valores de tus variables con las mias y veo que no hay difrencias antes de inyectar fallos

## Método 1

In [18]:
error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/buffer_weights/error_mask_054')
locs = load_obj('Data/Fault Characterization/variante_mask_vc_707/buffer_weights/locs_054')
#print('losc pesos',locs_pesos[0:10])
#loss: 3.0441 - accuracy: 0.1173

activation_aging = False


wfrac_size = 11
wint_size  = 4

In [14]:

#locs[0] = 0
#error_mask[0] = 'xx1xxxxxxxxxxxxx'
#

#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net3 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net3.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net3, frac_bits = wfrac_size, int_bits = wint_size)
for layer in Net3.layers:
    # weights contendrá una lista de arreglos, cada elemento de la lista contiene los pesos de un mismo tipo
    # por ejemplo en una capa convolucional la lista es [w,b] donde w son los pesos y b los bias (tambien son pesos)
    weights = layer.get_weights()
    if weights:
        print('Introducciendo errores en',layer.name)
        # La idea a seguir ahora es:
        # 1) Unir todos los pesos de la capa en una lista 1-D que simboliza su orden en memoria
        # 2) aplicar los fallos en esta lista 1-D
        # 3) retomar las dimensiones originales de los pesos e insertarlos en la capa.
        vectorized_weights = []
        # etapa 1) 
        for itm in weights:
            print('itm dentro')
            
            print('    Numero de pesos:',itm.size)
            vectorized_weights.extend(itm.flatten(order='F'))
            print('tamaño del vector vectorizado',len(vectorized_weights))
            print(type(vectorized_weights))
        vectorized_weights = np.array(vectorized_weights)
        #break
        print('shape', vectorized_weights.shape)
        print('  Numero total de pesos:',vectorized_weights.size)
        # etapa 2) 
        indexList,Listfault,NumberOfFaults = GenerateFaultsList(shape=vectorized_weights.shape,locs=locs,error_mask=error_mask)
        print('  Total de direcciones con fallos:',NumberOfFaults)
        print('  indexList:',len(indexList))
        print('  Listfault fallos:',len(Listfault))
        if NumberOfFaults > 0:
            faulty_weights = IntroduceFaultsInWeights(vectorized_weights,indexList,Listfault,wint_size,wfrac_size)
            #break
        # etapa 3)
        print('  Total de direcciones afectadas:',np.count_nonzero(vectorized_weights!=faulty_weights))
        diferentes=np.not_equal(vectorized_weights, faulty_weights)
        print('total de elementos iguales',  np.count_nonzero(np.equal(vectorized_weights, faulty_weights)))
        for index, itm in enumerate(weights):
            print('index',index)
            weights[index] = faulty_weights[0:itm.size].numpy().reshape(itm.shape,order='F')
            #faulty_weights = faulty_weights[itm.size:]
            layer.set_weights(weights)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net3.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
loss,acc  = Net3.evaluate(test_dataset)          

Introducciendo errores en Conv1
itm dentro
    Numero de pesos: 34848
tamaño del vector vectorizado 34848
<class 'list'>
itm dentro
    Numero de pesos: 96
tamaño del vector vectorizado 34944
<class 'list'>
shape (34944,)
  Numero total de pesos: 34944
  Total de direcciones con fallos: 466
  indexList: 466
  Listfault fallos: 466
inyectando errores en los pesos
  Total de direcciones afectadas: 249
total de elementos iguales 34695
index 0
index 1
Introducciendo errores en batch_normalization_45
itm dentro
    Numero de pesos: 96
tamaño del vector vectorizado 96
<class 'list'>
itm dentro
    Numero de pesos: 96
tamaño del vector vectorizado 192
<class 'list'>
itm dentro
    Numero de pesos: 96
tamaño del vector vectorizado 288
<class 'list'>
itm dentro
    Numero de pesos: 96
tamaño del vector vectorizado 384
<class 'list'>
shape (384,)
  Numero total de pesos: 384
  Total de direcciones con fallos: 2
  indexList: 2
  Listfault fallos: 2
inyectando errores en los pesos
  Total de direc

# Método 2

In [19]:


#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
for index,layer in enumerate(Net2.layers):
   
    
    weights_tmp=[]
    itm=[]
    #print(Net2.layers[index].__class__.__name__)
        
    weights = layer.get_weights()
    if weights:
        print(Net2.layers[index].__class__.__name__)
        if layer.__class__.__name__ in ['Conv2D','Conv1D','DepthwiseConv2D']:
            print('tamaño d ela capa' ,layer.output_shape)
            
            weights, biases = Net2.layers[index].get_weights()
            print(type(weights))
                
            w_shape= weights.shape
            b_shape= biases.shape
            print('shape wights_conv_antes',w_shape)
            print('shape bias_antes_conv',b_shape)
            wsize=weights.size
            weights_conv = weights.flatten(order='F')
            print(weights_conv.shape)
            bias=biases.flatten(order='F')
            itm= np.concatenate((weights_conv, biases))
            print(len(itm))
            #break
            
            print('itm.shape',itm.shape)
            positionList,faultList,NumberOfFaults = GenerateFaultsList(shape=itm.shape,locs=locs,error_mask=error_mask)
            print('  Total de direcciones con fallos:',NumberOfFaults)
            print('  indexList:',len(positionList))
            print('  Listfault fallos:',len(faultList))
            itm_fail= IntroduceFaultsInWeights(itm,positionList,faultList,wfrac_size,wint_size) 
           
            print('numero de fallos insertados', NumberOfFaults)
            print('tamaño de del array que contiene los weights',wsize)
            #Analizo la cantidad de posiciones que estarán afectadas contando los elementos menores 
            print('cantidad de pocisiones afectada capa',np.count_nonzero(positionList <= wsize))
            print('tamaño de la lista d eposiciones', len(positionList))
            #d = itm_fail[:wsize]
            #Extraigo la cantidad de lementos que tenia el array de weights y le coloco la form aoriginal para hacer el set
            w=np.reshape(itm_fail[:wsize], (w_shape))
            comp_weights = np.not_equal(w, weights)
            # para analizar la cantidad de elementos que han variado luego de introducir fallos
            print('cantidad de elementos diferente realmente', np.count_nonzero(itm_fail!=itm))
            print('cantidad de elementos iguales', np.count_nonzero(np.equal(itm_fail, itm)))
             #Extraigo la cantidad de lementos que tenia el array del bias y le coloco la form original para hacer el set
            b=np.reshape(itm_fail[wsize:], (b_shape))
            print('shape bias_conv',b.shape)
            comp_bias = np.not_equal(b, biases)
            #Compruno los elementos que han variado ante sy despues de los fallos
            print('Bias cantidad de elementos iguales', )
            print('Bias cantidad de elementos diferentes', np.sum(comp_bias == 0))
            #Lo inserto en un alista para formar la estructura que tenia luego del get_w
            weights_tmp.append(w)
            weights_tmp.append(b)
            Net2.layers[index].set_weights(weights_tmp)
            print('set weights')
            #Este código debe ir identado con el for pero quise ir viendo el acc por cada capa xq alex lo sugirió y lo coloqué 
            #aquí, para el modelo completo ya sabes cambia la posición 
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
loss,acc  = Net2.evaluate(test_dataset)                
print(Net2)

Conv2D
tamaño d ela capa (None, 55, 55, 96)
<class 'numpy.ndarray'>
shape wights_conv_antes (11, 11, 3, 96)
shape bias_antes_conv (96,)
(34848,)
34944
itm.shape (34944,)
  Total de direcciones con fallos: 466
  indexList: 466
  Listfault fallos: 466
inyectando errores en los pesos
numero de fallos insertados 466
tamaño de del array que contiene los weights 34848
cantidad de pocisiones afectada capa 466
tamaño de la lista d eposiciones 466
cantidad de elementos diferente realmente 464
cantidad de elementos iguales 34480
shape bias_conv (96,)
Bias cantidad de elementos iguales
Bias cantidad de elementos diferentes 96
set weights
BatchNormalization
Conv2D
tamaño d ela capa (None, 27, 27, 256)
<class 'numpy.ndarray'>
shape wights_conv_antes (5, 5, 96, 256)
shape bias_antes_conv (256,)
(614400,)
614656
itm.shape (614656,)
  Total de direcciones con fallos: 7277
  indexList: 7277
  Listfault fallos: 7277
inyectando errores en los pesos
numero de fallos insertados 7277
tamaño de del array que

# Metdodo optimizado

In [15]:


#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net, frac_bits = wfrac_size, int_bits = wint_size)
for layer in Net.layers:
    weights = layer.get_weights()
    if weights:
        print('Introduciendo errores en',layer.name)
        vectorized_weights = []
        for weight in weights:
            print('    - Numero de pesos:',weight.size)
            vectorized_weights.extend(weight.flatten(order='F'))
            print('    - Tamaño del vector vectorizado',len(vectorized_weights))

        vectorized_weights = np.array(vectorized_weights)
        print('  Numero total de pesos:',vectorized_weights.size)
        
        indexList,faultList,NumberOfFaults = GenerateFaultsList(shape=vectorized_weights.shape,locs=locs,error_mask=error_mask)
        print('  Total de direcciones con fallos:',NumberOfFaults)

        if NumberOfFaults > 0:
            faulty_weights = IntroduceFaultsInWeights(vectorized_weights,indexList,faultList,wint_size,wfrac_size)

            for index, weight in enumerate(weights):
                weights[index] = faulty_weights[0:weight.size].numpy().reshape(weight.shape,order='F')

            layer.set_weights(weights)
            
            print('  Total de direcciones afectadas:',np.count_nonzero(vectorized_weights!=faulty_weights))

        print('----------------------------------')

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
Net.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
loss,acc  = Net3.evaluate(test_dataset)          

Introduciendo errores en Conv1
    - Numero de pesos: 34848
    - Tamaño del vector vectorizado 34848
    - Numero de pesos: 96
    - Tamaño del vector vectorizado 34944
  Numero total de pesos: 34944
  Total de direcciones con fallos: 466
inyectando errores en los pesos
  Total de direcciones afectadas: 249
----------------------------------
Introduciendo errores en batch_normalization_50
    - Numero de pesos: 96
    - Tamaño del vector vectorizado 96
    - Numero de pesos: 96
    - Tamaño del vector vectorizado 192
    - Numero de pesos: 96
    - Tamaño del vector vectorizado 288
    - Numero de pesos: 96
    - Tamaño del vector vectorizado 384
  Numero total de pesos: 384
  Total de direcciones con fallos: 2
inyectando errores en los pesos
  Total de direcciones afectadas: 0
----------------------------------
Introduciendo errores en Conv2
    - Numero de pesos: 614400
    - Tamaño del vector vectorizado 614400
    - Numero de pesos: 256
    - Tamaño del vector vectorizado 614656
 

# Analizar salidas d elas variables para los dos casos

### antes de inyectar los fallos los vectores que pasamos son iguales

In [9]:
print('cantidad de elementos diferente yami', np.count_nonzero(vectorized_weights!=itm))
print('cantidad de elementos iguales yami', np.count_nonzero(np.equal(vectorized_weights, itm)))




cantidad de elementos diferente yami 0
cantidad de elementos iguales yami 34944
  Total de direcciones afectadas nicolas: 0


### luego ya no lo son

In [13]:
print('Elementos diferentes entre ambos vectores luego de inyect fallos', np.count_nonzero(faulty_weights!=itm_fail))
print('cantidad de elementos iguales vectores luego de inyect fallos', np.count_nonzero(np.equal(faulty_weights, itm_fail)))




print('  Total de direcciones  luego deinyec fallos nicolas:',np.count_nonzero(vectorized_weights!=faulty_weights))
print('cantidad de elementos diferente yami', np.count_nonzero(itm!=itm_fail))

Elementos diferentes entre ambos vectores luego de inyect fallos 461
cantidad de elementos iguales vectores luego de inyect fallos 34483
  Total de direcciones  luego deinyec fallos nicolas: 249
cantidad de elementos diferente yami 464


In [5]:

#Máscara base Buffer de pesos
error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/buffer_weights/error_mask_054')
locs = load_obj('Data/Fault Characterization/variante_mask_vc_707/buffer_weights/locs_054')
#print('losc pesos',locs_pesos[0:10])
#loss: 3.0441 - accuracy: 0.1173

activation_aging = False


wfrac_size = 11
wint_size  = 4



#Acá la creamos, notese que como no se introduciran fallos en activaciones no es necesario pasar locs ni masks
Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, aging_active=activation_aging,
                             word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)
Net2.load_weights(wgt_dir).expect_partial()
#Cuantizacion de los pesos
WeightQuantization(model = Net2, frac_bits = wfrac_size, int_bits = wint_size)
for index,layer in enumerate(Net2.layers):
   
    
    weights_tmp=[]
    #print(Net2.layers[index].__class__.__name__)
        
    if(len(Net2.layers[index].weights) > 0):
        print(Net2.layers[index].__class__.__name__)
        if layer.__class__.__name__ in ['Conv2D','Conv1D','DepthwiseConv2D']:
            print('tamaño d ela capa' ,layer.output_shape)
            weights, biases = Net2.layers[index].get_weights()
            w_shape= weights.shape
            b_shape= biases.shape
            print('shape wights_conv_antes',w_shape)
            print('shape bias_antes_conv',b_shape)
            wsize=weights.size
            weights_conv = np.reshape(weights, (-1))
            weig_conv_bias=np.concatenate((weights_conv, biases))
            itm= weig_conv_bias
            print('itm.shape',itm.shape)
            positionList,faultList,NumberOfFaults = GenerateFaultsList(shape=itm.shape,locs=locs,error_mask=error_mask)
            print('positionList',len(positionList))
            print('faultList',len(faultList))
            print(NumberOfFaults,NumberOfFaults)
            itm_fail= IntroduceFaultsInWeights(itm,positionList,faultList,wfrac_size,wint_size) 
            print('numero de fallos insertados', NumberOfFaults)
            print('tamaño de del array que contiene los weights',wsize)
            #Analizo la cantidad de posiciones que estarán afectadas contando los elementos menores 
            print('cantidad de pocisiones afectada capa',np.count_nonzero(positionList <= wsize))
            print('tamaño de la lista d eposiciones', len(positionList))
            
            #Extraigo la cantidad de lementos que tenia el array de weights y le coloco la form aoriginal para hacer el set
            w=np.reshape(itm_fail[:wsize], (w_shape))
            comp_weights = np.equal(w, weights)
            # para analizar la cantidad de elementos que han variado luego de introducir fallos
            print('cantidad de elementos iguales', np.count_nonzero(comp_weights))
            print('cantidad de elementos diferentes', np.sum(comp_weights == 0))
             #Extraigo la cantidad de lementos que tenia el array del bias y le coloco la form original para hacer el set
            b=np.reshape(itm_fail[wsize:], (b_shape))
            print('shape bias_conv',b.shape)
            comp_bias = np.equal(b, biases)
            #Compruno los elementos que han variado ante sy despues de los fallos
            print('Bias cantidad de elementos iguales', np.count_nonzero(comp_bias))
            print('Bias cantidad de elementos diferentes', np.sum(comp_bias == 0))
            #Lo inserto en un alista para formar la estructura que tenia luego del get_w
            weights_tmp.append(w)
            weights_tmp.append(b)
            Net2.layers[index].set_weights(weights_tmp)
            print('set weights')
            #Este código debe ir identado con el for pero quise ir viendo el acc por cada capa xq alex lo sugirió y lo coloqué 
            #aquí, para el modelo completo ya sabes cambia la posición 
            loss = tf.keras.losses.CategoricalCrossentropy()
            optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
            Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
            loss,acc  = Net2.evaluate(test_dataset)                
print(Net2)
#Net_faulty = WeightsFaults(Net2,locs,error_mask,wfrac_size,wint_size)

        

Conv2D
tamaño d ela capa (None, 55, 55, 96)
shape wights_conv_antes (11, 11, 3, 96)
shape bias_antes_conv (96,)
itm.shape (34944,)
positionList 466
faultList 466
466 466
inyectando errores en los pesos
numero de fallos insertados 466
tamaño de del array que contiene los weights 34848
cantidad de pocisiones afectada capa 466
tamaño de la lista d eposiciones 466
cantidad de elementos iguales 34385
cantidad de elementos diferentes 463
shape bias_conv (96,)
Bias cantidad de elementos iguales 96
Bias cantidad de elementos diferentes 0
set weights
47/47 [==============================] - 14s 300ms/step - loss: 9.9115 - accuracy: 0.1173
BatchNormalization
Conv2D
tamaño d ela capa (None, 27, 27, 256)
shape wights_conv_antes (5, 5, 96, 256)
shape bias_antes_conv (256,)
itm.shape (614656,)
positionList 7277
faultList 7277
7277 7277
inyectando errores en los pesos
numero de fallos insertados 7277
tamaño de del array que contiene los weights 614400
cantidad de pocisiones afectada capa 7275
tamaño 